# Ironbox: Agent Core and Framework Selection

This notebook demonstrates how the Agent Core processes queries and selects the appropriate framework for handling different types of requests.

## Overview

The Agent Core is the central component of the Ironbox system that:
1. Receives user queries
2. Uses the Framework Selector to determine the best framework for handling the query
3. Routes the query to the selected framework
4. Returns the response to the user

The Framework Selector can choose from four frameworks:
- **Route Framework**: For simple categorizable queries that can be handled by specialized agents
- **React Framework**: For reasoning and action problems that require tool use
- **Plan Framework**: For complex multi-step problems that require planning
- **Direct LLM**: For simple informational questions that can be answered directly

Let's explore how this works in practice.

## Setup

First, let's import the necessary modules and initialize the Agent Core.

In [ ]:
import os
import sys
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Import Ironbox components
from ironbox.core.agent_core import AgentCore
from ironbox.core.agent_framework import AgentFramework, RouteAgentFramework, ReactAgentFramework, PlanAgentFramework

# Initialize the Agent Core
agent_core = AgentCore()

# Setup is complete
print("Agent Core initialized successfully.")

## Framework Selection

Let's examine how the Framework Selector decides which framework to use for different types of queries.

In [ ]:
# Function to demonstrate framework selection
def demonstrate_framework_selection(query):
    print(f"Query: {query}")
    framework_type = agent_core.framework_selector.select_framework(query)
    print(f"Selected Framework: {framework_type}\n")
    return framework_type

# Test with different types of queries
queries = [
    "Register my Kubernetes cluster at endpoint https://k8s.example.com",  # Route Framework (Cluster Register)
    "How many pods are running in my production namespace?",               # React Framework (requires tool use)
    "Create a deployment plan for migrating my application to Kubernetes", # Plan Framework (complex multi-step)
    "What is Kubernetes?"                                                 # Direct LLM (informational)
]

for query in queries:
    demonstrate_framework_selection(query)

## Processing Queries with Different Frameworks

Now let's see how each framework processes a query.

In [ ]:
# Function to process a query using the Agent Core
def process_query(query):
    print(f"Processing query: {query}")
    response = agent_core.process_query(query, session_id="demo-session")
    print(f"Response: {response['response']}")
    print(f"Framework used: {response['framework']}\n")
    return response

# Process each query
for query in queries:
    process_query(query)

## Route Framework in Detail

Let's take a closer look at how the Route Framework works with specialized agents.

In [ ]:
# Get the Route Framework
route_framework = agent_core.frameworks.get("route")

# List available specialized agents
print("Available Specialized Agents:")
for agent_type, agent in agent_core.agents.items():
    print(f"- {agent_type}")

# Process a query with the Route Framework
route_query = "Check the health of my production cluster"
print(f"\nProcessing query with Route Framework: {route_query}")
from ironbox.core.agent_core import AgentState
state = AgentState(query=route_query, session_id="demo-session")
result_state = route_framework.process(state)
print(f"Response: {result_state.response}")
print(f"Agent used: {result_state.metadata.get('agent_used', 'Unknown')}")

## React Framework in Detail

Now let's examine how the React Framework uses tools to solve problems.

In [ ]:
# Get the React Framework
react_framework = agent_core.frameworks.get("react")

# List available tools
print("Available Tools:")
for tool_name in agent_core.tools.keys():
    print(f"- {tool_name}")

# Process a query with the React Framework
react_query = "Scale my frontend deployment to 3 replicas"
print(f"\nProcessing query with React Framework: {react_query}")
state = AgentState(query=react_query, session_id="demo-session")
result_state = react_framework.process(state)
print(f"Response: {result_state.response}")
print("\nThinking steps:")
for i, step in enumerate(result_state.metadata.get('steps', [])):
    print(f"Step {i+1}:")
    print(f"  Thought: {step.thought}")
    print(f"  Action: {step.action}")
    print(f"  Action Input: {step.action_input}")
    print(f"  Observation: {step.observation}\n")

## Plan Framework in Detail

Finally, let's see how the Plan Framework creates and executes plans for complex problems.

In [ ]:
# Get the Plan Framework
plan_framework = agent_core.frameworks.get("plan")

# Process a query with the Plan Framework
plan_query = "Create a new namespace, deploy a Redis instance, and configure it for high availability"
print(f"Processing query with Plan Framework: {plan_query}")
state = AgentState(query=plan_query, session_id="demo-session")
result_state = plan_framework.process(state)
print(f"Response: {result_state.response}")

# Display the plan and execution results
print("\nPlan:")
for i, step in enumerate(result_state.metadata.get('plan', [])):
    status = "✓" if step.completed else "✗"
    print(f"{status} Step {i+1}: {step.description}")
    if step.result:
        print(f"   Result: {step.result}")

## Conclusion

In this notebook, we've explored how the Agent Core and Framework Selection system works in Ironbox:

1. The Agent Core receives user queries and uses the Framework Selector to determine the best framework for handling them.
2. The Route Framework routes simple categorizable queries to specialized agents.
3. The React Framework uses a thinking-action-observation loop to solve reasoning and action problems.
4. The Plan Framework creates and executes plans for complex multi-step problems.
5. Direct LLM responses handle simple informational questions.

This flexible architecture allows Ironbox to handle a wide range of query types efficiently, using the most appropriate approach for each situation.